In [1]:
!pip install requests

In [3]:
API_KEY = "9WdGBWrQRU7rDR685nLCDgu79JEuuJ09NfZxxYJy"

import requests
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-06&api_key={API_KEY}"
response = requests.get(url)
response

<Response [200]>

In [5]:
data = response.json()


In [115]:
asteroids_data = []
target = 100

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-15&end_date=2024-01-20&api_key={API_KEY}"

response = requests.get(url)
data = response.json()
details = data['near_earth_objects']
while len(asteroids_data) < target:
      for date,ast_details in details.items():
          for ast in ast_details:
            asteroids_data.append(dict(
                id = int(ast['id']), # convert format accordingly
                name = ast['name'],
                magnitude = ast['absolute_magnitude_h'],
                dia_min = ast['estimated_diameter']['kilometers']['estimated_diameter_min']

            ))

            if len(asteroids_data) == target:
                break
          if len(asteroids_data) == target:
                break
      url = data['links']['next']
data['element_count']

101

In [123]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

API_KEY = "9WdGBWrQRU7rDR685nLCDgu79JEuuJ09NfZxxYJy"  # Replace with your actual NASA API key
target = 10000
page_size = 100
total_pages = target // page_size

def fetch_page(page_num):
    url = f"https://api.nasa.gov/neo/rest/v1/neo/browse?page={page_num}&size={page_size}&api_key={API_KEY}"
    response = requests.get(url)
    data = response.json()
    asteroids = []

    for ast in data.get('near_earth_objects', []):
        try:
            close_approach = ast['close_approach_data'][0] if ast['close_approach_data'] else None
            if close_approach:
                asteroids.append(dict(
                    id=int(ast['id']),
                    name=ast['name'],
                    magnitude=float(ast['absolute_magnitude_h']),
                    dia_min=float(ast['estimated_diameter']['kilometers']['estimated_diameter_min']),
                    dia_max=float(ast['estimated_diameter']['kilometers']['estimated_diameter_max']),
                    neo_reference_id=int(ast['neo_reference_id']),
                    is_potentially_hazardous_asteroid=ast['is_potentially_hazardous_asteroid'],
                    relative_velocity=float(close_approach['relative_velocity']['kilometers_per_hour']),
                    astronomical=float(close_approach['miss_distance']['astronomical']),
                    miss_distance_km=float(close_approach['miss_distance']['kilometers']),
                    miss_distance_lunar=float(close_approach['miss_distance']['lunar']),
                    orbiting_body=close_approach['orbiting_body'],
                    close_approach_date=datetime.strptime(close_approach['close_approach_date'], '%Y-%m-%d').strftime('%d %B %Y')
                ))
        except (KeyError, IndexError, ValueError, TypeError) as e:
            print(f"Skipped malformed asteroid data: {e}")

    return asteroids

asteroids_data = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_page, i) for i in range(total_pages)]
    for future in as_completed(futures):
        asteroids_data.extend(future.result())

print(f"Collected {len(asteroids_data)} asteroid records.")


Collected 1877 asteroid records.


In [125]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

API_KEY = "9WdGBWrQRU7rDR685nLCDgu79JEuuJ09NfZxxYJy"  # Replace this with your actual NASA API key
target = 10000
page_size = 100
max_workers = 10

def fetch_and_filter(page_num):
    url = f"https://api.nasa.gov/neo/rest/v1/neo/browse?page={page_num}&size={page_size}&api_key={API_KEY}"
    response = requests.get(url)
    data = response.json()
    asteroids = []

    for ast in data.get('near_earth_objects', []):
        try:
            if not ast['close_approach_data']:
                continue  # Skip if no approach data

            ca_data = ast['close_approach_data'][0]
            asteroid = {
                "id": int(ast['id']),
                "name": ast['name'],
                "magnitude": float(ast['absolute_magnitude_h']),
                "dia_min": float(ast['estimated_diameter']['kilometers']['estimated_diameter_min']),
                "dia_max": float(ast['estimated_diameter']['kilometers']['estimated_diameter_max']),
                "neo_reference_id": int(ast['neo_reference_id']),
                "is_potentially_hazardous_asteroid": ast['is_potentially_hazardous_asteroid'],
                "relative_velocity": float(ca_data['relative_velocity']['kilometers_per_hour']),
                "astronomical": float(ca_data['miss_distance']['astronomical']),
                "miss_distance_km": float(ca_data['miss_distance']['kilometers']),
                "miss_distance_lunar": float(ca_data['miss_distance']['lunar']),
                "orbiting_body": ca_data['orbiting_body'],
                "close_approach_date": datetime.strptime(ca_data['close_approach_date'], '%Y-%m-%d').strftime('%d %B %Y')
            }
            asteroids.append(asteroid)
        except (KeyError, IndexError, ValueError, TypeError) as e:
            continue  # Skip malformed entries

    return asteroids

asteroids_data = []
page = 0

while len(asteroids_data) < target:
    print(f"Fetching pages {page} to {page + max_workers - 1}...")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_and_filter, p) for p in range(page, page + max_workers)]
        for future in as_completed(futures):
            asteroids_data.extend(future.result())
            if len(asteroids_data) >= target:
                break
    page += max_workers  # Move to the next batch of pages

print(f"✅ Collected {len(asteroids_data)} valid asteroid records.")

Fetching pages 0 to 9...
Fetching pages 10 to 19...
Fetching pages 20 to 29...
Fetching pages 30 to 39...
Fetching pages 40 to 49...
Fetching pages 50 to 59...
Fetching pages 60 to 69...
Fetching pages 70 to 79...
Fetching pages 80 to 89...
Fetching pages 90 to 99...
Fetching pages 100 to 109...
Fetching pages 110 to 119...
Fetching pages 120 to 129...
Fetching pages 130 to 139...
Fetching pages 140 to 149...
Fetching pages 150 to 159...
Fetching pages 160 to 169...
Fetching pages 170 to 179...
Fetching pages 180 to 189...
Fetching pages 190 to 199...
Fetching pages 200 to 209...
Fetching pages 210 to 219...
Fetching pages 220 to 229...
Fetching pages 230 to 239...
Fetching pages 240 to 249...
Fetching pages 250 to 259...
Fetching pages 260 to 269...
Fetching pages 270 to 279...
Fetching pages 280 to 289...
Fetching pages 290 to 299...
Fetching pages 300 to 309...
Fetching pages 310 to 319...
Fetching pages 320 to 329...
Fetching pages 330 to 339...
Fetching pages 340 to 349...
Fetchin

In [69]:
data['links']

{'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-06&end_date=2024-01-11&detailed=false&api_key=9WdGBWrQRU7rDR685nLCDgu79JEuuJ09NfZxxYJy',
 'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-12-27&end_date=2024-01-01&detailed=false&api_key=9WdGBWrQRU7rDR685nLCDgu79JEuuJ09NfZxxYJy',
 'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-06&detailed=false&api_key=9WdGBWrQRU7rDR685nLCDgu79JEuuJ09NfZxxYJy'}

In [137]:
len(asteroids_data)

10013

In [9]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [75]:
#used previously but the data is incorrect

from datetime import datetime
asteroids_data = []
target = 10000
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-06&api_key={API_KEY}"

response = requests.get(url)
data = response.json()
details = data['near_earth_objects']
while len(asteroids_data) < target:
      for date,ast_details in details.items():
          for ast in ast_details:  
              
            asteroids_data.append(dict(
                id = int(ast['id']), # convert format accordingly
                name = ast['name'],
                magnitude = float(ast['absolute_magnitude_h']),
                dia_min = float(ast['estimated_diameter']['kilometers']['estimated_diameter_min']),
                dia_max = float(ast['estimated_diameter']['kilometers']['estimated_diameter_max']),
                neo_reference_id = int(ast['neo_reference_id']),
                is_potentially_hazardous_asteroid = ast['is_potentially_hazardous_asteroid'],
                relative_velocity = float(ast['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
                astronomical =float(ast['close_approach_data'][0]['miss_distance']['astronomical']),
                miss_distance_km = float(ast['close_approach_data'][0]['miss_distance']['kilometers']),
                miss_distance_lunar = float(ast['close_approach_data'][0]['miss_distance']['lunar']),
                orbiting_body = ast['close_approach_data'][0]['orbiting_body'],
                close_approach_date = datetime.strptime(ast['close_approach_data'][0]['close_approach_date'], '%Y-%m-%d').strftime('%d %B %Y')
                                
            ))

            if len(asteroids_data) == target:
                break
          if len(asteroids_data) == target:
                break
      url = data['links']['next']

In [77]:
len(asteroids_data)

10000

In [79]:
!pip install mysql-connector-python sqlalchemy pymysql

In [81]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# Connection info
username = 'root'
password = 'Ramya@123'
host = '127.0.0.1'
port = 3306
database = 'sys'

# Encode password to handle special characters like '@'
encoded_password = quote_plus(password)

# Create the engine
engine = create_engine(f"mysql+pymysql://{username}:{encoded_password}@{host}:{port}/{database}")

In [43]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS asteroids (
    ID BIGINT ,
    NAME VARCHAR(100),
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN
)
""")

In [47]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS close_approach (

    neo_reference_id BIGINT,
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR(50)
    
)
""")

In [49]:
import mysql.connector

# Replace these values with your actual MySQL credentials
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Ramya@123',           # your MySQL password
    database='sys'   # replace with your DB name
)

# Create a cursor object
cursor = conn.cursor()

In [131]:
from datetime import datetime

insert_query = """
INSERT INTO asteroids (
    ID  ,
    NAME ,
    absolute_magnitude_h ,
    estimated_diameter_min_km ,
    estimated_diameter_max_km ,
    is_potentially_hazardous_asteroid 
)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for asteroid in asteroids_data:

    values = (
        asteroid['id'],
        asteroid['name'],
        asteroid['magnitude'],
        asteroid['dia_min'],
        asteroid['dia_max'],
        asteroid['is_potentially_hazardous_asteroid']
    )
    cursor.execute(insert_query, values)

conn.commit()
print("Data inserted successfully!")

Data inserted successfully!


In [133]:
from datetime import datetime

insert_query = """
INSERT INTO close_approach (
   neo_reference_id ,
    close_approach_date ,
    relative_velocity_kmph ,
    astronomical ,
    miss_distance_km ,
    miss_distance_lunar ,
    orbiting_body 
)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

for asteroid in asteroids_data:
    # Convert '01 May 2025' to '2025-05-01'
    date_obj = datetime.strptime(asteroid['close_approach_date'], '%d %B %Y')
    formatted_date = date_obj.strftime('%Y-%m-%d')

    values = (
        asteroid['neo_reference_id'],
        formatted_date,
        asteroid['relative_velocity'],
        asteroid['astronomical'],
        asteroid['miss_distance_km'],
        asteroid['miss_distance_lunar'],
        asteroid['orbiting_body']
        
    )
    cursor.execute(insert_query, values)

conn.commit()
print("Data inserted successfully!")

Data inserted successfully!


In [143]:
# Clear any previous result sets
while cursor.nextset():
    pass
# Execute and fetch the new result
cursor.execute("SELECT * FROM asteroids")
rows = cursor.fetchall()

# Display results using pandas
import pandas as pd
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,ID,NAME,absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid
0,2007977,7977 (1977 QQ5),15.16,2.469190,5.521280,0
1,2008013,8013 Gordonmoore (1990 KA),17.09,1.015210,2.270070,0
2,2008014,8014 (1990 MF),18.88,0.445200,0.995497,1
3,2008034,8034 Akka (1992 LR),18.02,0.661538,1.479240,0
4,2008035,8035 (1992 TB),17.12,1.001280,2.238930,0
...,...,...,...,...,...,...
10008,3648841,(2013 TP4),27.30,0.009216,0.020608,0
10009,3648843,(2013 TQ4),27.40,0.008801,0.019681,0
10010,3648844,(2013 TR4),23.24,0.059780,0.133672,0
10011,3648846,(2013 TS4),20.90,0.175612,0.392681,0


In [127]:
cursor.execute("""
DELETE from asteroids
""")

In [141]:
cursor.execute("""
desc asteroids
""")

In [129]:
cursor.execute("""
DELETE FROM close_approach
""")